# Using xrft

`xrft` is a package to do Fourier transforms of `xarray` data easily. One can use the `numpy`-like functions, which are not exact (phase and amplitude shifts), but deliver the same results as a `numpy`-FT would do. There is also the possibility to correct these shifts with `xrft`. Another handy advantage of `xrft` is that the visualization of the results can be done with `HoloViews` in a very simple manner.

(Note the generation of each plot takes about 5 secs., you can add the `dynamic=True` parameter, if you want to load the plots faster. This is thoroughly explained in the [holoviews tutorial](holoviews_with_postopus.ipynb))

In [ ]:
import xrft

%config InlineBackend.figure_formats = ['svg']


import holoviews as hv

from postopus.octopus_run import Run

hv.extension("bokeh", "matplotlib")  # Allow for interactive plots

input file is already defined in the folder (s. GitLab repo), otherwise we recommend defining it in the notebook

In [ ]:
cd ../octopus_data/interference/

Assuming you have octopus in your PATH:

In [ ]:
!octopus > out_gs.log 2>&1

In [ ]:
run = Run(".")

## Original Data

In [ ]:
xa = run.Maxwell.td.b_field(source="z=0").vx

In [ ]:
xa

### Visualization with `HoloViews`

In [ ]:
hv_dst_orig = hv.Dataset(xa)
hv_imt_orig = hv_dst_orig.to(hv.Image, ["x", "y"])
hv.output(max_frames=3000)
hv_imt_orig.opts(
    colorbar=True, width=500, height=400, cmap="seismic", clim=(-(10**-3), 10**-3)
)

## Numpy-like fft

In [ ]:
# `xa.drop_vars("step")` is needed here as "step" and "t" represent the same dimension.
# Fxa = xrft.fft(xa.drop_vars("step"), dim="t") # numpy.fft-like behaviour
Fxa = xrft.fft(xa.drop_vars("step"), dim="t", real_dim="t")  # numpy.rfft-like behaviour

In [ ]:
Fxa

### Visualization with `HoloViews`

In [ ]:
Fxa.name = "testFT"  # needed to use holoviews
hv_dst = hv.Dataset(Fxa.real)
hv_imt = hv_dst.to(hv.Image, ["x", "y"])

In [ ]:
hv_imt.opts(
    colorbar=True, width=500, height=400, cmap="seismic", clim=(-(10**-3), 10**-3)
)

## Inverse transform

There is a shift when inverse transforming.

In [ ]:
# iFxa = xrft.ifft(Fxa, dim="freq_t") # Inverse of np.fft
iFxa = xrft.ifft(Fxa, dim="freq_t", real_dim="freq_t")  # Inverse of np.rfft

### Visualization with `HoloViews`

In [ ]:
iFxa.name = "iFtest"
hv_dst_ift = hv.Dataset(iFxa.real)
hv_imt_ift = hv_dst_ift.to(hv.Image, ["x", "y"])
hv.output(max_frames=3000)
hv_imt_ift.opts(
    colorbar=True, width=500, height=400, cmap="seismic", clim=(-(10**-3), 10**-3)
)

## FFT with true amplitude and true phase

This is good for the cases in which the data is not centered at 0, see https://xrft.readthedocs.io/en/latest/DFT-iDFT_example.html. As one can see we go back to the original data after the inverse transform. There is no shift.

In [ ]:
Fxa2 = xrft.fft(xa.drop_vars("step"), dim="t", true_amplitude=True, true_phase=True)

### Visualization with `HoloViews`

In [ ]:
Fxa2.name = "testFT2"
hv_dst2 = hv.Dataset(Fxa2.real)
hv_imt_2 = hv_dst2.to(hv.Image, ["x", "y"])
hv_imt_2.opts(
    colorbar=True, width=500, height=400, cmap="seismic", clim=(-(10**-3), 10**-3)
)

## Inverse Transform

In [ ]:
iFxa2 = xrft.ifft(Fxa2, dim="freq_t", true_amplitude=True, true_phase=True)

### Visualization with `HoloViews`

In [ ]:
iFxa2.name = "iFtest2"
hv_dst_ift2 = hv.Dataset(iFxa2.real)
hv_imt_ift2 = hv_dst_ift2.to(hv.Image, ["x", "y"])
hv_imt_ift2.opts(
    colorbar=True, width=500, height=400, cmap="seismic", clim=(-(10**-3), 10**-3)
)

## Spectral density

In [ ]:
sd = xrft.power_spectrum(xa.drop_vars("step"), dim="t", real_dim="t", scaling="density")

### Visualization with `HoloViews`

In [ ]:
sd.name = "Spectral_density"
hv_dst_sd = hv.Dataset(sd)
hv_imt_sd = hv_dst_sd.to(hv.Image, ["x", "y"])
hv_imt_sd.opts(colorbar=True, width=500, height=400, cmap="seismic")